# Running Tile2Net on Colab

If you don't have access to a physical GPU on your computer, or your GPU is not powerful enough, you can use the following workflow to use the tool!

There are two ways to use Tile2Net - via the command line or via imported python modules. Using a Colab workflow, the first way is a bit more straightforward, but both are possible.

You will notice exclamation points `!` and percent symbols `%` at the front of many code blocks in this notebook. Both are Colab syntax used to execute bash commands and "magic" commands in the colab environment. The relevant magic commands will be explained as they come up.

## Command Line Interface

### Step 0: Mount Google Drive

This step is optional, and is only necessary if you plan to upload your own tiles to the tool.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### Step 1: Clone the Repository

In [ ]:
!git clone https://github.com/VIDA-NYU/tile2net.git

Once the repository has cloned, you can use the magic `%cd` command to go into the resulting    tile2net `directory`. It is important to use the magic command rather than calling `!cd`, because the latter creates a temporary subshell in which the command is called, whereas the former will persist the directory change.

In [ ]:
!cd tile2net
!pwd

In [ ]:
%cd tile2net
!pwd

Always make sure you're up to date with the latest version of the code!

In [ ]:
!git pull

### Step 2: Activate a Virtual Environment and Install Dependencies

When using Colab, it is not necessary to create a conda environment, because each session creates its own environment. The following command installs all of the dependencies present in the [requirements-dev.txt](https://github.com/VIDA-NYU/tile2net/blob/main/requirements-dev.txt) file (That's what the period at the end does!)

In [ ]:
import os

repo_path = '/content/tile2net'
pyproject_path = os.path.join(repo_path, 'pyproject.toml')

if os.path.exists(pyproject_path):
    with open(pyproject_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    modified = False
    for line in lines:
        # Corrected: search for 'requires-python' instead of 'python_requires'
        if line.strip().startswith('requires-python'):
            # Modify the line to be more permissive, including Python 3.12
            new_lines.append('requires-python = ">=3.10, <3.13"\n')
            modified = True
        else:
            new_lines.append(line)

    if modified:
        with open(pyproject_path, 'w') as f:
            f.writelines(new_lines)
        print(f"Modified {pyproject_path} to allow Python 3.12.")
    else:
        print(f"Could not find 'requires-python' in {pyproject_path} or it was already permissive (should not happen with the given error).")
else:
    print(f"Error: {pyproject_path} not found. Please ensure the tile2net repository is cloned.")

# Original installation command
# In Colab, the following two lines are unnecessary, but they are very important when working locally
# !conda create --name testenv python==3.11
# !conda activate testenv
!python -m pip install -e .

### Step 3: Start Generating a Network!

You're pretty much done! All you need to do is call the same bash commands from the quick start command line tutorial, but with a `!` in front.

You can call `!python -m tile2net generate --help` or `!python -m tile2net inference --help` for more details on the generate and inference functions, respectively.

For example:

In [ ]:
!python -m tile2net generate -l "42.35555189953313, -71.07168915322092, 42.35364837213307, -71.06437423368418" -n example -o "content/output"  | python -m tile2net inference

### Step 4: Explore and Save Your Results

Now that you've generated your stitched tiles and created your pedestrian network, it's time to explore!

Given that your output directory is `output_dir`, you will see the following folders:

- `output_dir/tiles/static`: the raw, unstitched orthorectified tiles used during segmentation
- `output_dir/tiles/stitched`: the stitched tiles
- `output_dir/segmentation/*`: metadata about the segmentation process used
- `output_dir/polygons/*`: shapefiles of polygons created from the segmentation results
- `output_dir/network/*`: the final pedestrian network shapefiles, ready to be visualized in any GIS software!

In Colab, you can download these files using the `!zip` command:

In [ ]:
%cd content/output
!zip -r example.zip example

from google.colab import files
files.download('example.zip')

## Python Modules



To use Google Colab with the python module version of tile2net, the process is almost identical to the process described in the quick start module tutorial, with one caveat: you must restart your runtime after installing your libraries, but before running any commands that may require those libraries

First, follow steps 0-2 as directed in the [Command Line Interface](#command-line-interface) section.

**Then, go to `Runtime>Restart Session` or use the shortcut `CTRL+M .`**

After this, you can use the tile2net module as the Quick Start Module tutorial describes

In [3]:
import tile2net

location = "41.010361394056375, -74.14833643482527, 40.949516438466226, -74.06819594124879"
raster = tile2net.Raster(
    location=location,
    # name='boston common',
    output_dir='./example_dir',
    zoom=19
)
raster.Project

INFO       Geocoding (40.979938916261304, -74.10826618803702), this may take a while...
INFO       Geocoded '(40.979938916261304, -74.10826618803702)' to
	'76, North Irving Street, Ridgewood, Bergen County, New Jersey, 07450, United States'
INFO       Using NewJersey as the source at location=(40.9495164385, -74.1483364348, 41.0103613941, -74.0681959412)
INFO       Using base_tilesize=256 from source


tile2net.raster.project.Project

# Running Tile2Net on Colab


In [4]:
import tile2net

location = "41.010361394056375, -74.14833643482527, 40.949516438466226, -74.06819594124879"
raster = tile2net.Raster(
    location=location,
    # name='boston common',
    output_dir='./example_dir',
    zoom=19
)
raster.Project

INFO       Geocoding (40.979938916261304, -74.10826618803702), this may take a while...
INFO       Geocoded '(40.979938916261304, -74.10826618803702)' to
	'76, North Irving Street, Ridgewood, Bergen County, New Jersey, 07450, United States'
INFO       Using NewJersey as the source at location=(40.9495164385, -74.1483364348, 41.0103613941, -74.0681959412)
INFO       Using base_tilesize=256 from source


tile2net.raster.project.Project